# Multimodal RAG usage

This notebooks, goes over the usage of the provided [multimodal rag src module](../src/multimodal.py) amd some query examples, to understand the details of the implementation please refer to the [multimodal rag notebook](./2_multimodal_rag.ipynb) 

In [16]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.multimodal_rag import MultimodalRetrieval

## Instantiate the MultimodalRetrieval module

In [2]:
multimodal = MultimodalRetrieval()

## Parse input document

In [ ]:
filepath = os.path.join(kit_dir,"data/sample_docs/invoicesample.pdf")
raw_pdf_elements, output_path = multimodal.extract_pdf(filepath)

## Process parsing outputs

In [4]:
text_docs, table_docs, image_paths = multimodal.process_raw_elements(raw_pdf_elements, output_path)

## Create a vectorstore

In [5]:
retriever = multimodal.create_vectorstore()

2024-09-23 16:23:22,519 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-09-23 16:23:25,351 [INFO] - Use pytorch device: cpu


max_seq_length  512


/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/src/multimodal_rag.py:236: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


In [6]:
retriever = multimodal.vectorstore_ingest(retriever, text_docs, table_docs, image_paths, summarize_texts=True, summarize_tables=True)

## Using summaries of images to get a final response

In [7]:
multimodal.set_retrieval_chain(retriever, image_retrieval_type="summary")

In [8]:
multimodal.call("how many apples they bought")

{'question': 'how many apples they bought',
 'answer': '\n\nAccording to the provided context, they bought 1 apple.',
 'source_documents': [Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02qNpDtsFmWzjvKt/3Q0idzTAu49be9VO+uYW0u80FYZybwlnh31PItwWlsuEFPfSpyzPv6kqqusyjpqG1uN6gdcyIUsexVrzWJ4CVUZBqvBsz7OedvExvoX11sB6q9+WPABl8Tn1341s7jjeHoXdlGqml4USVCTG1kO8OsHBf/lwtC+52k90tu7qqoNtV1ws65pt9H1JMsGjGCBOYmPLbEn0h+AXXM/tQmDfdtHcwzLudF5Hh7axHtf3hxeWP6NdfXs/z7h+6Tbld1vp3TwRRoXGiwZYFgUUxAVsSAibEKaU1UEf42TxpWE2L5JSS2syS34+NrNligTS1OAksjP/miWFEsCf25Dv0Uz/AHqXDOO1R6Nt+QGPKyBZZN8j33VjWCnkakA3pkEafuh2CNuUavWjTm8nu0dgH28KIgj0kN40owjDbOO1rdDl1/gFxg/Z211k0QkCjL3YPaXwzAfLTME7jBsFgM2rLA6ePd5uGzm4WRrQ+tWlMY5nSozAlBPm47vphQV3aodwm/QbbeYsnCosqp6kQDCtnSmERJ7BphMcsisY445k5Y7Mbqmoy9+58

In [9]:
multimodal.call("whats the address of the store")

{'question': 'whats the address of the store',
 'answer': '\n\nAccording to the provided context, the address of the store is 123 Somewhere St, Melbourne VIC 3000.',
 'source_documents': [Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02qNpDtsFmWzjvKt/3Q0idzTAu49be9VO+uYW0u80FYZybwlnh31PItwWlsuEFPfSpyzPv6kqqusyjpqG1uN6gdcyIUsexVrzWJ4CVUZBqvBsz7OedvExvoX11sB6q9+WPABl8Tn1341s7jjeHoXdlGqml4USVCTG1kO8OsHBf/lwtC+52k90tu7qqoNtV1ws65pt9H1JMsGjGCBOYmPLbEn0h+AXXM/tQmDfdtHcwzLudF5Hh7axHtf3hxeWP6NdfXs/z7h+6Tbld1vp3TwRRoXGiwZYFgUUxAVsSAibEKaU1UEf42TxpWE2L5JSS2syS34+NrNligTS1OAksjP/miWFEsCf25Dv0Uz/AHqXDOO1R6Nt+QGPKyBZZN8j33VjWCnkakA3pkEafuh2CNuUavWjTm8nu0dgH28KIgj0kN40owjDbOO1rdDl1/gFxg/Z211k0QkCjL3YPaXwzAfLTME7jBsFgM2rLA6ePd5uGzm4WRrQ+tWlMY5nSozAlBPm47vphQV3aodwm/Qbb

In [10]:
multimodal.call("what is the main color in the business logo")

{'question': 'what is the main color in the business logo',
 'answer': '\n\nAccording to the context, the main colors in the Sunny Farm business logo are gold and yellow.',
 'source_documents': [Document(metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}, page_content='The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, giving the impression of a sunny day. The logo is placed on a white background, making it stand out.'),
  Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02

In [11]:
multimodal.call("what is written in the logo")

{'question': 'what is written in the logo',
 'answer': '\n\nI do not have information regarding this. The context only describes the design and colors of the logo, but it does not mention what is written in the logo.',
 'source_documents': [Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02qNpDtsFmWzjvKt/3Q0idzTAu49be9VO+uYW0u80FYZybwlnh31PItwWlsuEFPfSpyzPv6kqqusyjpqG1uN6gdcyIUsexVrzWJ4CVUZBqvBsz7OedvExvoX11sB6q9+WPABl8Tn1341s7jjeHoXdlGqml4USVCTG1kO8OsHBf/lwtC+52k90tu7qqoNtV1ws65pt9H1JMsGjGCBOYmPLbEn0h+AXXM/tQmDfdtHcwzLudF5Hh7axHtf3hxeWP6NdfXs/z7h+6Tbld1vp3TwRRoXGiwZYFgUUxAVsSAibEKaU1UEf42TxpWE2L5JSS2syS34+NrNligTS1OAksjP/miWFEsCf25Dv0Uz/AHqXDOO1R6Nt+QGPKyBZZN8j33VjWCnkakA3pkEafuh2CNuUavWjTm8nu0dgH28KIgj0kN40owjDbOO1rdDl1/gFxg/Z211k0QkCjL3YPaXwzAfLTME7jBsF

## Using raw images to get a final response

In [12]:
multimodal.set_retrieval_chain(retriever, image_retrieval_type="raw")

In [13]:
multimodal.call("how many apples they bought")

{'question': 'how many apples they bought',
 'answer': '\n\nAccording to the provided context, they bought 1 apple.',
 'source_documents': [Document(metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}, page_content='The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, giving the impression of a sunny day. The logo is placed on a white background, making it stand out.'),
  Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02qNpDtsFmWzjvKt/3Q0idzTAu49be9VO+uYW0u80FYZybwlnh31PItwW

In [14]:
multimodal.call("what is the main color in the business logo")

{'question': 'what is the main color in the business logo',
 'answer': '\n\nThe main color in the business logo is yellow.',
 'source_documents': [Document(metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}, page_content='The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, giving the impression of a sunny day. The logo is placed on a white background, making it stand out.'),
  Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02qNpDtsFmWzjvKt/3Q0idzTAu49be9VO+uYW0u80FYZybwlnh

In [15]:
multimodal.call("what is written in the logo")

{'question': 'what is written in the logo',
 'answer': '\n\nThe logo says "Sunny Farm Victoria."',
 'source_documents': [Document(metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}, page_content='The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, giving the impression of a sunny day. The logo is placed on a white background, making it stand out.'),
  Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:43:38', 'page_number': 1, 'orig_elements': 'eJzNV1uL3DYY/SvCtNCWkdHNuuxbnlpouw1k8lCWZdHl06zAY09suc02zX+v7PEmSzNtNoXZLgwYndGxpXO+78i+eldBC3vo8k0K1QWqGho0815jAoRhEV2DjYoSA+Miek5BAq02qNpDtsFmWzjvKt/3Q0idzTAu49be9VO+uYW0u80FYZybwlnh31PItwWlsuEFPfSpyzPv6kqqus